In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

import time
import operator
import re

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

##  데이터를 읽어온다

In [2]:
df = pd.read_csv('data10/review_data.csv')
df


,score,review,y
0,5,친절하시고 깔끔하고 좋았습니다,1
1,5,조용하고 고기도 굿,1
2,4,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",1
3,4,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,1
4,5,고기도 맛있고 서비스는 더 최고입니다~,1
...,...,...,...
540,3,추웟어요 고기 외에는 별로에요..,0
541,1,고기질과 육전은 좋다.다만 한우손님 돼지고기 손님을 차별한다(돼지손님은 주차불가.네...,0
542,5,직접 구워주시고 진짜맛있음. 반찬도 맛있음. 직원분이 친절하게 잘해주시네요,1
543,4,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ,1


###  한글 외의 모든 글자를 제거한다.

In [3]:
def text_cleaning(text) :
    # 한글 정규식
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    # 한글 정규식에 해당하지 않는 것은 길이가 0인 문자열로 변환한다.
    result = hangul.sub('', text)
    return result

In [4]:
# 리뷰 내용을 정제한다.
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(x))
# review를 제거한다
df

,score,review,y,ko_text
0,5,친절하시고 깔끔하고 좋았습니다,1,친절하시고 깔끔하고 좋았습니다
1,5,조용하고 고기도 굿,1,조용하고 고기도 굿
2,4,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",1,갈비탕과 냉면 육회비빔밥이 맛있습니다
3,4,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,1,대체적으로 만족하나와인의 구성이 살짝 아쉬움
4,5,고기도 맛있고 서비스는 더 최고입니다~,1,고기도 맛있고 서비스는 더 최고입니다
...,...,...,...,...
540,3,추웟어요 고기 외에는 별로에요..,0,추웟어요 고기 외에는 별로에요
541,1,고기질과 육전은 좋다.다만 한우손님 돼지고기 손님을 차별한다(돼지손님은 주차불가.네...,0,고기질과 육전은 좋다다만 한우손님 돼지고기 손님을 차별한다돼지손님은 주차불가네이버예...
542,5,직접 구워주시고 진짜맛있음. 반찬도 맛있음. 직원분이 친절하게 잘해주시네요,1,직접 구워주시고 진짜맛있음 반찬도 맛있음 직원분이 친절하게 잘해주시네요
543,4,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ,1,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ


###  형태소 분석

In [7]:
from konlpy.tag import Okt

# konlpy 라이브러리로 텍스트 데이터에서 형태소를 추출한다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    result = []
    
    for a1 in pos :
        result.append(f'{a1[0]}/{a1[1]}')
    
    return result
    
get_pos(df['ko_text'][0])

['친절하시고/Adjective', '깔끔하고/Adjective', '좋았습니다/Adjective']

## 분류 모델의 학습 데이터로 변환하기

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# 형태소를 벡터 형태의 학습 데이터 셋으로 변환한다.
index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())
X

<545x3030 sparse matrix of type '<class 'numpy.int64'>'
	with 9692 stored elements in Compressed Sparse Row format>

In [9]:
index_vectorizer.vocabulary_

{'친절하시고/Adjective': 2647,
 '깔끔하고/Adjective': 428,
 '좋았습니다/Adjective': 2403,
 '조용하고/Adjective': 2356,
 '고기/Noun': 233,
 '도/Josa': 721,
 '굿/Noun': 330,
 '갈비탕/Noun': 120,
 '과/Josa': 260,
 '냉면/Noun': 528,
 '육회/Noun': 2065,
 '비빔밥/Noun': 1419,
 '이/Josa': 2082,
 '맛있습니다/Adjective': 1013,
 '대/Modifier': 671,
 '체적/Noun': 2604,
 '으로/Josa': 2067,
 '만족하나/Adjective': 956,
 '와인/Noun': 1996,
 '의/Josa': 2077,
 '구성/Noun': 293,
 '살짝/Noun': 1476,
 '아쉬움/Noun': 1705,
 '맛있고/Adjective': 1001,
 '서비스/Noun': 1508,
 '는/Josa': 589,
 '더/Noun': 701,
 '최고/Noun': 2613,
 '입니다/Adjective': 2182,
 '가/Josa': 24,
 '입/Noun': 2177,
 '에서/Josa': 1897,
 '녹아요/Verb': 553,
 '였습니다/Verb': 1935,
 '살살/Noun': 1473,
 '녹는/Verb': 552,
 '최상급/Noun': 2615,
 '소고기/Noun': 1540,
 '를/Josa': 901,
 '맛/Noun': 995,
 '보고왔습니다/Verb': 1316,
 '구워주고/Verb': 302,
 '가성/Noun': 73,
 '비/Noun': 1408,
 '짱/Noun': 2528,
 '콜키/Noun': 2688,
 '지/Josa': 2477,
 '프리/Noun': 2788,
 '라서/Josa': 885,
 '을/Josa': 2070,
 '가지/Noun': 89,
 '고/Josa': 227,
 '가면/Noun': 61,
 '좋은/Adjective

In [19]:
print(df['ko_text'][0])
print(X[0])

친절하시고 깔끔하고 좋았습니다
  (0, 428)	0.7573091319965198
  (0, 2403)	0.4011484274975501
  (0, 2647)	0.5153278739898709


### TF_IDF 로 변환한다.

In [18]:
# 위에서 만든 형태소 벡터를 학습 데이터 벡터로 생성한다.
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)
print(X[0])

  (0, 428)	0.7573091319965198
  (0, 2403)	0.4011484274975501
  (0, 2647)	0.5153278739898709


## 학습데이터를 생성한다.

In [20]:
from sklearn.model_selection import train_test_split

# 결과
y = df['y']
# 학습용과 검증용으로 나눈다.
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(x_train.shape)
print(x_test.shape)

(381, 3030)
(164, 3030)


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 로지스틱 회귀 모델을 생성한다.
lr= LogisticRegression()
# 학습
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# 테스트용 데이터를 통해 예측한다.
y_pred = lr.predict(x_test)

In [24]:
# 평가한다.
r1 = accuracy_score(y_test, y_pred)
r2 = precision_score(y_test, y_pred)
r3 = recall_score(y_test, y_pred)
r4 = f1_score(y_test, y_pred)
r5 = roc_auc_score(y_test, y_pred)

In [26]:
print(f' accuracy  : {r1}')
print(f' precision : {r2}')
print(f' recall    : {r3}')
print(f' f1        : {r4}')
print(f' roc_auc   : {r5}')

 accuracy  : 0.9207317073170732
 precision : 0.9207317073170732
 recall    : 1.0
 f1        : 0.9587301587301587
 roc_auc   : 0.5


## 중요 피처 형태소

In [32]:
# 회귀 모델의 계수를 높은 순서대로 정렬한다.
# sorted((value, index) for index, value in enumerate(lr.coef_[0]))

# 계수를 (인덱스, 계수) 형태로 만들어준다.
a1 = list(enumerate(lr.coef_[0]))
# display(a1)
# sorted 함수를 쓰면 첫 번째 것을 기준으로 정렬 하기 때문에
# index, 계수 형태를, index형태로 바꿔준다.\
a2 = list(((value, index) for index, value in a1))
# a2
# 정렬한다.
coef_pos_index = sorted(a2, reverse =True)
coef_pos_index

[(-1.3208982964630704, 1309),
 (-0.7981607549974865, 2069),
 (-0.6786514679459582, 901),
 (-0.6348125218422457, 2070),
 (-0.6243608121564568, 2427),
 (-0.5908742050530668, 2198),
 (-0.5642647374828045, 18),
 (-0.5642647374828045, 1608),
 (-0.559570558312224, 2289),
 (-0.55807899727516, 2371),
 (-0.5487429281621264, 338),
 (-0.5175217544446736, 16),
 (-0.511457640209706, 330),
 (-0.48757632618933294, 1054),
 (-0.47990020254502586, 589),
 (-0.4729706853889953, 2312),
 (-0.4632089602832897, 2620),
 (-0.46105306019977005, 2242),
 (-0.45869346982955034, 354),
 (-0.45869346982955034, 2127),
 (-0.4513177882723517, 1156),
 (-0.4470648796225434, 1260),
 (-0.43610592171944607, 1430),
 (-0.4306810841928304, 2082),
 (-0.42890319914486325, 278),
 (-0.42890319914486325, 1349),
 (-0.4182616307504133, 390),
 (-0.4073031725071584, 1384),
 (-0.4035681468821034, 1720),
 (-0.3972398699685608, 2862),
 (-0.39646983231469823, 2344),
 (-0.39501205230479186, 397),
 (-0.392457571836005, 1025),
 (-0.388249463168

In [34]:
# 단어 번호 : 단어 형태의 딕셔너리를 생성한다.
text_data_dict = {}

for text, text_id in index_vectorizer.vocabulary_.items() :
    text_data_dict[text_id] = text

text_data_dict


{2647: '친절하시고/Adjective',
 428: '깔끔하고/Adjective',
 2403: '좋았습니다/Adjective',
 2356: '조용하고/Adjective',
 233: '고기/Noun',
 721: '도/Josa',
 330: '굿/Noun',
 120: '갈비탕/Noun',
 260: '과/Josa',
 528: '냉면/Noun',
 2065: '육회/Noun',
 1419: '비빔밥/Noun',
 2082: '이/Josa',
 1013: '맛있습니다/Adjective',
 671: '대/Modifier',
 2604: '체적/Noun',
 2067: '으로/Josa',
 956: '만족하나/Adjective',
 1996: '와인/Noun',
 2077: '의/Josa',
 293: '구성/Noun',
 1476: '살짝/Noun',
 1705: '아쉬움/Noun',
 1001: '맛있고/Adjective',
 1508: '서비스/Noun',
 589: '는/Josa',
 701: '더/Noun',
 2613: '최고/Noun',
 2182: '입니다/Adjective',
 24: '가/Josa',
 2177: '입/Noun',
 1897: '에서/Josa',
 553: '녹아요/Verb',
 1935: '였습니다/Verb',
 1473: '살살/Noun',
 552: '녹는/Verb',
 2615: '최상급/Noun',
 1540: '소고기/Noun',
 901: '를/Josa',
 995: '맛/Noun',
 1316: '보고왔습니다/Verb',
 302: '구워주고/Verb',
 73: '가성/Noun',
 1408: '비/Noun',
 2528: '짱/Noun',
 2688: '콜키/Noun',
 2477: '지/Josa',
 2788: '프리/Noun',
 885: '라서/Josa',
 2070: '을/Josa',
 89: '가지/Noun',
 227: '고/Josa',
 61: '가면/Noun',
 2410: '좋은/Adj

In [44]:
# 긍전단위 상위 20개
coef_pos_index[:20]

[(-1.3208982964630704, 1309),
 (-0.7981607549974865, 2069),
 (-0.6786514679459582, 901),
 (-0.6348125218422457, 2070),
 (-0.6243608121564568, 2427),
 (-0.5908742050530668, 2198),
 (-0.5642647374828045, 18),
 (-0.5642647374828045, 1608),
 (-0.559570558312224, 2289),
 (-0.55807899727516, 2371),
 (-0.5487429281621264, 338),
 (-0.5175217544446736, 16),
 (-0.511457640209706, 330),
 (-0.48757632618933294, 1054),
 (-0.47990020254502586, 589),
 (-0.4729706853889953, 2312),
 (-0.4632089602832897, 2620),
 (-0.46105306019977005, 2242),
 (-0.45869346982955034, 354),
 (-0.45869346982955034, 2127)]

In [36]:
# 부정단어 상위 20개
coef_pos_index[-20:]

[(0.24679667032241231, 2502),
 (0.24891995032921144, 2403),
 (0.25524269807615496, 1007),
 (0.27497412556325945, 1137),
 (0.2815164205401264, 2330),
 (0.28357909077357324, 1388),
 (0.2877593252630111, 2247),
 (0.30505768671388145, 873),
 (0.31589910537944854, 961),
 (0.316510602995349, 1029),
 (0.329628912112967, 2388),
 (0.3321764632892218, 1030),
 (0.33338168071876506, 1006),
 (0.3506221157234927, 2376),
 (0.37302990624830473, 1001),
 (0.3946025105538175, 2613),
 (0.42579808505548594, 999),
 (0.4512398765608644, 2246),
 (0.47694347109037805, 1093),
 (0.6670890308913168, 1017)]

In [45]:
# coef_pos_index 안 있는 값들 중 단어 번호를 실제 단어로 변환한다.
coef_pos_text = []

for value, index in coef_pos_index :
    # index 에 해당하는 단어를 추출한다.
    text = text_data_dict[index]
    # 계수와 단어의 조합으로 만들어 담아준다.
    coef_pos_text.append((value, text))
    
# 상위 20개
for text in coef_pos_text[:20]:
    print(text)

#하위 20개
print('----------------')
for text in coef_pos_text[-20:]:
    print(text)


(-1.3208982964630704, '별로/Noun')
(-0.7981607549974865, '은/Josa')
(-0.6786514679459582, '를/Josa')
(-0.6348125218422457, '을/Josa')
(-0.6243608121564568, '주문/Noun')
(-0.5908742050530668, '있는데/Adjective')
(-0.5642647374828045, 'ㅠㅠㅠㅠ/KoreanParticle')
(-0.5642647374828045, '시끄러워요/Adjective')
(-0.559570558312224, '적어요/Verb')
(-0.55807899727516, '종업원/Noun')
(-0.5487429281621264, '그/Determiner')
(-0.5175217544446736, 'ㅠㅠ/KoreanParticle')
(-0.511457640209706, '굿/Noun')
(-0.48757632618933294, '매우/Noun')
(-0.47990020254502586, '는/Josa')
(-0.4729706853889953, '점/Noun')
(-0.4632089602832897, '추웟어/Noun')
(-0.46105306019977005, '작아서/Adjective')
(-0.45869346982955034, '그럭저럭/Adverb')
(-0.45869346982955034, '이었지만/Verb')
----------------
(0.24679667032241231, '진짜/Noun')
(0.24891995032921144, '좋았습니다/Adjective')
(0.25524269807615496, '맛있는/Adjective')
(0.27497412556325945, '모두/Noun')
(0.2815164205401264, '정말/Noun')
(0.28357909077357324, '분위기/Noun')
(0.2877593252630111, '잘/VerbPrefix')
(0.30505768671388145, '